# Date Difference Calculator - MCP-Style Tool Test

## Purpose

This notebook demonstrates how an LLM can extend its capabilities beyond text generation by delegating **deterministic computation** to external tools.

## Why This Matters

### ❌ What LLMs Are BAD At:
- Precise arithmetic
- Date calculations (leap years, month boundaries)
- Deterministic computation
- Edge case handling

### ✅ What LLMs Are GOOD At:
- Understanding natural language questions
- Extracting structured information
- Orchestrating tool calls
- Formatting responses

## The MCP Pattern

```
User Query (Natural Language)
        ↓
    LLM Agent
    ├─ Understands intent
    ├─ Extracts dates
    └─ Calls Tool ──→ calculate_days_between()
                            ↓
                    Exact Computation
                    (Deterministic)
                            ↓
                    Returns Result
                            ↓
        LLM formats response
```

## Setup

In [ ]:
# Import the MCP-style tool
import sys
from pathlib import Path

# Add parent directory to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from mcp_tools.date_calculator import calculate_days_between, TOOL_METADATA

## Tool Metadata

In a full MCP implementation, the LLM would query available tools and see this metadata:

In [ ]:
import json
print("Tool Metadata:")
print("=" * 60)
print(json.dumps(TOOL_METADATA, indent=2))

## Test 1: Basic Date Difference

**Natural Language Question:**
> "How many days are there between January 1, 2024 and February 14, 2024?"

**What the LLM does:**
1. Understands the question is about date difference
2. Extracts: `start_date = "2024-01-01"`, `end_date = "2024-02-14"`
3. Calls the tool (does NOT try to calculate itself)
4. Formats the result in natural language

In [ ]:
# Simulate LLM extracting dates from natural language
user_question = "How many days are there between January 1, 2024 and February 14, 2024?"

# LLM extracts structured inputs
start_date = "2024-01-01"
end_date = "2024-02-14"

print(f"User Question: {user_question}")
print(f"\nExtracted Dates:")
print(f"  Start: {start_date}")
print(f"  End: {end_date}")

# LLM calls the tool
print(f"\n[Tool Call] calculate_days_between('{start_date}', '{end_date}')")
result = calculate_days_between(start_date, end_date)

# Show raw tool output
print(f"\n[Tool Result]")
print(json.dumps(result, indent=2))

# LLM formats the response
if result['success']:
    days = result['number_of_days']
    print(f"\n[LLM Response]")
    print(f"There are {days} days between {start_date} and {end_date}.")
else:
    print(f"\n[Error] {result['error']}")

## Test 2: Leap Year Edge Case

**Natural Language Question:**
> "How many days from February 28, 2024 to March 1, 2024?"

**Why the LLM needs a tool:**
- 2024 is a leap year (Feb has 29 days)
- LLM might forget this and say "1 day" (incorrect)
- Tool guarantees correct answer: **2 days**

In [ ]:
user_question = "How many days from February 28, 2024 to March 1, 2024?"

start_date = "2024-02-28"
end_date = "2024-03-01"

print(f"User Question: {user_question}")
print(f"\n⚠️  Edge Case: 2024 is a leap year! Feb has 29 days.")
print(f"    An LLM might forget this and calculate incorrectly.\n")

result = calculate_days_between(start_date, end_date)

if result['success']:
    days = result['number_of_days']
    print(f"[Tool Result] {days} days")
    print(f"\n[LLM Response]")
    print(f"There are {days} days between Feb 28 and Mar 1, 2024.")
    print(f"(Note: 2024 is a leap year, so Feb 29 exists!)")

## Test 3: Negative Days (Reverse Order)

**Natural Language Question:**
> "How many days from December 31, 2024 back to January 1, 2024?"

**Tool behavior:**
- Returns negative number when end_date < start_date
- LLM can interpret this meaningfully

In [ ]:
user_question = "How many days from December 31, 2024 back to January 1, 2024?"

start_date = "2024-12-31"
end_date = "2024-01-01"

print(f"User Question: {user_question}\n")

result = calculate_days_between(start_date, end_date)

if result['success']:
    days = result['number_of_days']
    print(f"[Tool Result] {days} days\n")
    print(f"[LLM Response]")
    print(f"Going backward from Dec 31 to Jan 1 is {abs(days)} days in the past.")
    print(f"(The negative value indicates the end date is before the start date.)")

## Test 4: Error Handling - Invalid Format

**Natural Language Question:**
> "How many days between 01/15/2024 and 01/20/2024?"

**What happens:**
- User provides dates in MM/DD/YYYY format
- Tool expects ISO 8601 (YYYY-MM-DD)
- Tool returns clear error message
- LLM can ask user to reformat

In [ ]:
user_question = "How many days between 01/15/2024 and 01/20/2024?"

# LLM might extract dates as-is (wrong format)
start_date = "01/15/2024"  # ❌ Wrong format
end_date = "01/20/2024"    # ❌ Wrong format

print(f"User Question: {user_question}\n")

result = calculate_days_between(start_date, end_date)

print(f"[Tool Result]")
print(json.dumps(result, indent=2))

if not result['success']:
    print(f"\n[LLM Response]")
    print(f"I need the dates in YYYY-MM-DD format.")
    print(f"Could you provide them as:")
    print(f"  - 2024-01-15 (instead of 01/15/2024)")
    print(f"  - 2024-01-20 (instead of 01/20/2024)")

## Test 5: Error Handling - Invalid Date

**Natural Language Question:**
> "How many days from February 30, 2024 to March 1, 2024?"

**What happens:**
- Feb 30 doesn't exist (Feb has max 29 days)
- Tool catches this and returns error
- LLM can explain the issue to user

In [ ]:
user_question = "How many days from February 30, 2024 to March 1, 2024?"

start_date = "2024-02-30"  # ❌ Invalid date (Feb has max 29 days)
end_date = "2024-03-01"

print(f"User Question: {user_question}\n")

result = calculate_days_between(start_date, end_date)

print(f"[Tool Result]")
print(json.dumps(result, indent=2))

if not result['success']:
    print(f"\n[LLM Response]")
    print(f"I notice an issue with your date: February 30, 2024 doesn't exist.")
    print(f"February 2024 has 29 days (it's a leap year).")
    print(f"Did you mean February 28 or 29?")

## Test 6: Complex Real-World Question

**Natural Language Question:**
> "If my project started on September 1, 2024 and I need to submit it by December 14, 2024, how many days do I have to work on it?"

**Full LLM + Tool workflow:**

In [ ]:
user_question = (
    "If my project started on September 1, 2024 and I need to submit it by "
    "December 14, 2024, how many days do I have to work on it?"
)

print(f"User Question: {user_question}\n")

# LLM extracts dates from natural language
print("[LLM Processing]")
print("  Detected: Date calculation needed")
print("  Extracted start: September 1, 2024 → 2024-09-01")
print("  Extracted end: December 14, 2024 → 2024-12-14")
print("  Tool selected: calculate_days_between\n")

start_date = "2024-09-01"
end_date = "2024-12-14"

result = calculate_days_between(start_date, end_date)

print(f"[Tool Call]")
print(f"  calculate_days_between('{start_date}', '{end_date}')\n")

print(f"[Tool Result]")
print(json.dumps(result, indent=2))

if result['success']:
    days = result['number_of_days']
    print(f"\n[LLM Response]")
    print(f"You have {days} days to work on your project.")
    print(f"\nBreakdown:")
    print(f"  - Start: September 1, 2024")
    print(f"  - Deadline: December 14, 2024")
    print(f"  - Total time: {days} days")
    print(f"  - That's approximately {days // 7} weeks and {days % 7} days")
    print(f"\nGood luck with your project! 🚀")

## Summary: Why MCP-Style Tools Matter

### Without Tools (Pure LLM):
```python
# LLM might respond:
"There are approximately 104 or 105 days..." ❌ Imprecise
"About 3.5 months, so roughly 105 days..." ❌ Estimation
"Let me think... September has 30 days..." ❌ Slow, error-prone
```

### With MCP Tool:
```python
# Tool returns:
result = calculate_days_between('2024-09-01', '2024-12-14')
# {'success': True, 'number_of_days': 104}

# LLM responds:
"You have exactly 104 days to work on your project." ✅ Exact
```

### Key Principles:

1. **LLMs orchestrate, tools compute**
   - LLM: Natural language understanding
   - Tool: Deterministic calculation

2. **No approximation, only precision**
   - Tool returns exact integers
   - No "about", "roughly", "approximately"

3. **Proper error handling**
   - Tool validates inputs
   - Returns clear error messages
   - LLM can guide user to fix issues

4. **Extends LLM capabilities**
   - LLM gains date arithmetic ability
   - Without needing to be retrained
   - Simply by having access to the tool

This is the future of AI: **Specialized tools + General intelligence = Powerful systems**

## Next Steps

To integrate this with a real LLM agent:

1. **Register the tool** with your LLM framework (LangChain, LlamaIndex, etc.)
2. **Define tool schema** so LLM knows when to call it
3. **Enable function calling** in your LLM API
4. **Let the LLM decide** when to use the tool vs. answering directly

Example with OpenAI function calling:

```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate_days_between",
            "description": "Calculate exact number of days between two dates",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_date": {
                        "type": "string",
                        "description": "Start date in YYYY-MM-DD format"
                    },
                    "end_date": {
                        "type": "string",
                        "description": "End date in YYYY-MM-DD format"
                    }
                },
                "required": ["start_date", "end_date"]
            }
        }
    }
]
```